In [0]:
%pip install nbformat databricks-sdk[openai]==0.38.0 dspy --quiet
dbutils.library.restartPython()

In [0]:
%run ../prompts/code_extraction_prompt 

In [0]:
%run ../helper/GradingModule

In [0]:
module = Module(databricks.sdk.WorkspaceClient(), extract_code = True)

In [0]:
candidate_dict = {
  "X": {
    "code_extraction_prompt": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-sql/sql_notebook_abhay",
    # "Optimization": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-optimization/Optimization-X",
    # "ML": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-ml/04-Machine-Learning-X"
  }
}

module_list = {
 "code_extraction_prompt": code_extraction_module
}

llm_models = {
  'llm-405B': 'databricks/databricks-meta-llama-3-1-405b-instruct',
  # 'llm-70B': 'databricks/databricks-meta-llama-3-3-70b-instruct'
}

In [0]:
import time 

model_dict = {}

for k,v in llm_models.items():
  start_time = time.time()
  # dspy.context(lm=dspy.LM(llm_models[k]))
  lm = dspy.LM(llm_models[k])
  dspy.settings.configure(lm=lm)
  # error_dict = {}
  answers_dict = {}
  for candidate, values in candidate_dict.items():
    for section, context_path in values.items():
      module.set_module_dict(module_list[section])
      results = module.get_error_and_answer_dict(context_path)
      results['answers_dict']['candidate'] = candidate
      results['answers_dict']['section'] = section
      results['answers_dict']['context'] = results['context'] 
      answers_dict[section] = results['answers_dict']
  end_time = time.time()
  execution_time = end_time - start_time
  print(f" time taken by model {k} for execution is {execution_time} seconds")
  model_dict[k] = {'answers_dict': answers_dict, 'execution_time': execution_time, 'lm':lm}

In [0]:
answers_df = pd.concat(model_dict['llm-405B']['answers_dict'].values(), ignore_index=True)
answers_df.display()